# PRIMAP-hist data Preparation

This jupyter notebook sets out a method for preparing PRIMAPhist data for plotting. It can further be used as a guide and basis for setting up other input datasets. 

The dataset can be viewed and accessed at:
https://www.pik-potsdam.de/paris-reality-check/primap-hist/

Gütschow, J.; Jeffery, L.; Gieseke, R. (2019): The PRIMAP-hist national historical emissions time series (1850-2016). v2.0. GFZ Data Services. https://doi.org/10.5880/pik.2019.001

The downloaded data should be stored in the 'input-data' folder of this repository. PRIMAP-hist is updated roughly once per year. The user should therefore check for updates at the above link and download new data when available. If this is done, the new filenames will need to be added below. 

------
Possible improvments:
*  unit conversion
* warn of over-writing files AND/OR automise file names.  Currently it's possible to give a file a poor name. Would require a look-up for category names though...


In [1]:
# import modules

# system
import re
import sys
import os

# data handling
import pandas as pd
import numpy as np

# open climate data packages
from countrygroups import UNFCCC, EUROPEAN_UNION, ANNEX_ONE, NON_ANNEX_ONE
from shortcountrynames import to_name

# global stocktake tools
import gst_tools.gst_utils as utils

In [32]:
## User definition of requirements

include_extrapolated_data = True

# to understand the codes to use here, please see the PRIMAP-hist documentation
raw_entity = 'CO2'
raw_sector = 'M.0.EL'
raw_scenario = 'HISTCR'

# choose something useful! These will be used to generate the new filename.
new_variable_name = 'CO2-total-excl-LU'
new_source_name = 'PRIMAP-hist_v2.3.1'

# Based on countrygroups package, select the group of countries you would like to extract. 
# Note that the raw data also includes groups.
needed_countries = UNFCCC

# First year of data needed for further plotting
start_year = 1990


In [34]:
## reduce the data

# select the right file 
# !*** Note that the file names here will need to be updated for new versions of PRIMAP-hist ***! 
if include_extrapolated_data:
    raw_data_file = 'Guetschow-et-al-2021-PRIMAP-hist_v2.3.1_20-Sep_2021.csv'
else:
    raw_data_file = 'Guetschow-et-al-2021-PRIMAP-hist_v2.3.1_no_extrap_20-Sep_2021.csv'

# get the data
raw_data_folder = 'input-data'
fname = os.path.join('', raw_data_folder, raw_data_file)
print('reading ' + fname)
raw_data = pd.read_csv(fname)

# LTN: rename column names to ensure they match columns names in these tools
raw_data.rename(columns={'scenario (PRIMAP-hist)': 'scenario',
                         'area (ISO3)': 'country',
                         'category (IPCC2006_PRIMAP)': 'category'}, inplace=True)

# reduce to only the desired variable (one per output file)
new_data = raw_data.loc[(raw_data['entity'] == raw_entity) &
                        (raw_data['scenario'] == raw_scenario) &
                        (raw_data['category'] == raw_sector)
                        ]

# reduce the countries or regions to only those desired
new_data = new_data.loc[new_data['country'].isin(needed_countries)]

# tell the user if any of the needed countries are missing and, if yes, which ones:
missing_countries = list(set(needed_countries) - set(new_data['country'].unique()))
if missing_countries:
    print('Not all countries requested were available in the raw data. You are missing the following:')
    for country in missing_countries:
        print('   ' + to_name(country))
    print('---------')

# reduce to only required years
new_data = utils.change_first_year(new_data, start_year)

# rename columns to follow conventions
new_data = new_data.rename(columns={'entity': 'variable'})

# make sure 'variable' contains all necessary information
new_data['variable'] = new_variable_name

# label the source
new_data['source'] = new_source_name

new_data = utils.check_column_order(new_data)

# check!
new_data

reading input-data\Guetschow-et-al-2021-PRIMAP-hist_v2.3.1_20-Sep_2021.csv
Not all countries requested were available in the raw data. You are missing the following:
   Palestine
   European Union
---------
First year of data available is now 1990
Last year of data available is 2019


,category,country,scenario,source,unit,variable,1990,1991,1992,1993,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,M.0.EL,AFG,HISTCR,PRIMAP-hist_v2.3.1,Gg CO2 / yr,CO2-total-excl-LU,2810.0,2640.0,1590.0,1520.0,...,8650.0,12400.0,10500.0,8830.0,8140.0,8230.0,7100.0,6860.0,7060.0,9780.0
1,M.0.EL,AGO,HISTCR,PRIMAP-hist_v2.3.1,Gg CO2 / yr,CO2-total-excl-LU,10800.0,11200.0,11600.0,12000.0,...,23500.0,24700.0,26500.0,26300.0,38000.0,29100.0,30300.0,29600.0,27600.0,27000.0
2,M.0.EL,ALB,HISTCR,PRIMAP-hist_v2.3.1,Gg CO2 / yr,CO2-total-excl-LU,4270.0,6050.0,4030.0,3920.0,...,5900.0,6660.0,6390.0,6590.0,7300.0,6220.0,6210.0,7160.0,6670.0,7090.0
3,M.0.EL,AND,HISTCR,PRIMAP-hist_v2.3.1,Gg CO2 / yr,CO2-total-excl-LU,409.0,417.0,426.0,436.0,...,556.0,525.0,523.0,512.0,497.0,497.0,504.0,506.0,494.0,470.0
4,M.0.EL,ARE,HISTCR,PRIMAP-hist_v2.3.1,Gg CO2 / yr,CO2-total-excl-LU,47000.0,52700.0,53300.0,61000.0,...,154000.0,162000.0,199000.0,197000.0,191000.0,211000.0,203000.0,171000.0,170000.0,170000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,M.0.EL,WSM,HISTCR,PRIMAP-hist_v2.3.1,Gg CO2 / yr,CO2-total-excl-LU,104.0,108.0,111.0,111.0,...,188.0,203.0,199.0,199.0,210.0,236.0,247.0,245.0,252.0,353.0
191,M.0.EL,YEM,HISTCR,PRIMAP-hist_v2.3.1,Gg CO2 / yr,CO2-total-excl-LU,9710.0,9320.0,10100.0,8810.0,...,22400.0,22800.0,23400.0,31700.0,30700.0,16400.0,13200.0,12700.0,20300.0,20600.0
192,M.0.EL,ZAF,HISTCR,PRIMAP-hist_v2.3.1,Gg CO2 / yr,CO2-total-excl-LU,274000.0,284000.0,262000.0,279000.0,...,447000.0,461000.0,443000.0,455000.0,485000.0,460000.0,464000.0,472000.0,456000.0,465000.0
193,M.0.EL,ZMB,HISTCR,PRIMAP-hist_v2.3.1,Gg CO2 / yr,CO2-total-excl-LU,2490.0,2450.0,2480.0,2530.0,...,2650.0,2880.0,3600.0,3880.0,4270.0,4360.0,4730.0,6500.0,7030.0,7090.0


In [35]:
## write the data to file

"""
First ensure that years, unit, 'country', and variable are all in data. If they are
can proceed to print data
"""

if 'country' not in new_data.columns or 'unit' not in new_data.columns:

    print('Missing required information! Please check your input data and processing!')

else:

    # define filename as composite of variable and source name
    fname_out = new_source_name + '_' + new_variable_name + '.csv'
    fullfname_out = os.path.join('proc-data', fname_out)

    # check folder exists
    if not os.path.exists('proc-data'):
        os.makedirs('proc-data')

    # write to csv in proc data folder
    new_data.to_csv(fullfname_out, index=False)

    # celebrate success 
    print('Processed data written to file! - ' + fullfname_out)


Processed data written to file! - proc-data\PRIMAP-hist_v2.3.1_CO2-total-excl-LU.csv


**Test ground below**

In [6]:
new_data.columns


Index(['category', 'country', 'scenario', 'source', 'unit', 'variable', '1990',
       '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999',
       '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016'],
      dtype='object')